In [2]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import os
import time 
from sklearn.preprocessing import OneHotEncoder

In [3]:
data = np.array(pd.read_csv("Datasets/digit-recognizer/train.csv"))
x_data, y_data = data[:, 1:data.shape[0]], data[:, 0:1]

indices = np.arange(len(x_data))
np.random.shuffle(indices)
x_data, y_data = x_data[indices], y_data[indices]

In [4]:
split = 0.9
idx = int(x_data.shape[0] * split)
trainx, trainy = x_data[:idx]/255., y_data[:idx]
x_validate, y_validate = x_data[idx:]/255., y_data[idx:]

In [5]:
class NeuralNetwork:
    def __init__(self, layers: list(tuple())) -> None:
        """ Input as: 
            NeuralNetwork(
                [
                    ({no of features}, activation='{function}'), ---> input layer
                    ({neurons/nodes}, activation='{function}')
                    .
                    .
                    .
                    ({neurons/nodes}, activation='{function}') -----> output layer
                ]
            )"""
        np.random.seed(1024)
        
        self.layers = layers
        self.activationFunc = [a for _, (_, a) in enumerate(self.layers)]
        self.num_layers = len(self.layers)
        self.params = {}
        
        for l in range(1, self.num_layers):
            self.params[f'w{l}'] = np.random.rand(self.layers[l][0], self.layers[l-1][0]) - 0.5
            self.params[f'b{l}'] = np.random.rand(self.layers[l][0], 1) - 0.5
    
    def _activation(self, func: str, z, backprop=False):
        if func == 'relu':
            if backprop:
                return z > 0
            return np.maximum(0, z)
        
        elif func == 'sigmoid':
            return 1 / (1 + np.exp(-z))
        
        elif func == 'softmax':
            exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
            return exp_z / np.sum(exp_z, axis=0, keepdims=True)
    
    def _loss(self, func, y_predicted, labels):
        labels = OneHotEncoder().fit_transform(labels).toarray().T
        if func == 'cross_entropy':
            _, n = labels.shape
            epsilon = 1e-15
            y_predicted = np.maximum(epsilon, y_predicted)
            loss = -1/n * np.sum(labels * np.log(y_predicted))
            return loss
    
    def _forward_prop(self, x):
        z_a = {}
        for l in range(self.num_layers-1):
            if l == 0:
                z_a[f"z{l+1}"] = self.params[f"w{l+1}"].dot(x) + self.params[f"b{l+1}"]
                z_a[f"a{l+1}"] = self._activation(self.activationFunc[l+1], z_a[f"z{l+1}"])
            else:
                z_a[f"z{l+1}"] = self.params[f"w{l+1}"].dot(z_a[f"a{l}"]) + self.params[f"b{l+1}"]
                z_a[f"a{l+1}"] = self._activation(self.activationFunc[l+1], z_a[f"z{l+1}"])
        
        return z_a
    
    def _back_prop(self, zanda, x, y):
        m, _ = x.shape
        gradients = {}
        dz = {}
        dz_prev = None
        for l in reversed(range(1, self.num_layers)):
            if l == self.num_layers - 1:
                dA = zanda[f"a{self.num_layers-1}"] - y
                dZ = dA
                gradients[f"dw{l}"] = 1 / m * dZ.dot(zanda[f"a{l-1}"].T)
                gradients[f"db{l}"] = 1 / m * np.sum(dZ)
                dz_prev = dZ
            
            elif l > 1:
                w = self.params[f"w{l+1}"]
                dA = w.T.dot(dz_prev)
                dZ = dA * self._activation(self.activationFunc[l], zanda[f"z{l}"], backprop=True)
                gradients[f"dw{l}"] = 1 / m * dZ.dot(zanda[f"z{l-1}"].T)
                gradients[f"db{l}"] = 1 / m * np.sum(dZ)
                dz_prev = dZ
            
            elif l == 1:
                dA = self.params[f"w{l+1}"].T.dot(dz_prev)
                dZ = dA * self._activation(self.activationFunc[l], zanda[f"z{l}"], backprop=True)
                gradients[f"dw{l}"] = 1 / m * dZ.dot(x.T)
                gradients[f"db{l}"] = 1 / m * np.sum(dZ)

        return gradients
    
    
    def fit(self, x_train, y_train, learning_rate, lossFunction, validation_data=None, epochs=10, batch_size=32):
        """ Train the neural network based on training data.
            Input as: model.fit(
                            x_train, y_train, 
                            epoch=n, 
                            learning_rate=alpha, 
                            validation_data=(x, y), 
                            batch_size=32
                        )"""
        m, _ = x_train.shape
        y_train_encoded = OneHotEncoder().fit_transform(y_train).toarray()
        sub_steps = m // batch_size
        
        for epoch in range(epochs):
            # --------------------------------------
            start_idx, end_idx = 0, batch_size
            print(f"\nEpoch {epoch+1}/{epochs}")
            indices = np.arange(len(x_train))
            np.random.shuffle(indices)
            x_train, y_train_encoded = x_train[indices], y_train_encoded[indices]
            y_train = y_train[indices]
            # -------------------------------------
            total_loss = 0
            total_samples = 0
            correct_predictions = 0
            for step in range(sub_steps+1):
                print(f"{step+1}/{sub_steps}", end="\r")
                # ------- transformations --------------------------------
                x, y = x_train[start_idx:end_idx].T, y_train_encoded[start_idx:end_idx].T
                yacc = y_train[start_idx:end_idx].T
                start_idx, end_idx = end_idx, end_idx+batch_size
                # --------------------------------------------------------
                # -------- training --------------------------------------
                za = self._forward_prop(x)
                gradients = self._back_prop(za, x, y)
                for params in self.params.keys():
                    self.params[params] -= learning_rate * gradients[f"d{params}"]
                # --------------------------------------------------------
                # ----------- loss and acc ------------------------------
            ## Calculating accuracy using validation data
            print(f"{sub_steps}/{sub_steps}", end=" ")
            validate = self._forward_prop(x_validate.T)
            accuracy = self.accuracy(validate[f"a{self.num_layers-1}"], y_validate.T)
            loss = self._loss(lossFunction, validate[f"a{self.num_layers-1}"], y_validate)
            print(f"val_acc: {accuracy} - val_loss: {loss}")
            
    def accuracy(self, y_pred, y):
        predictions = self.get_predictions(y_pred)
        acc = np.sum(predictions == y) / y.size
        return acc
        
    def get_predictions(self, y_pred):
        return np.argmax(y_pred, axis=0)

    def summary(self):
        for i, (nodes, activation) in enumerate(self.layers):
            print(f"Layer {i}:\n\tNodes: {nodes} \n\tActivation: {activation}")

In [6]:
model = NeuralNetwork(
    [
        (784, 'relu'),
        (256, 'relu'),
#         (128, 'relu'),
        (64, 'relu'),
        (10, 'softmax')
    ]
)

In [7]:
model.fit(
    trainx, trainy, 
    epochs=5, 
    learning_rate=0.1,  
    lossFunction='cross_entropy',
    batch_size=32,
    validation_data = (x_validate, y_validate)
)


Epoch 1/5
1181/1181 val_acc: 0.7983333333333333 - val_loss: 0.7487279260091708

Epoch 2/5
1181/1181 val_acc: 0.840952380952381 - val_loss: 0.5638374991908717

Epoch 3/5
1181/1181 val_acc: 0.8607142857142858 - val_loss: 0.48600685112653746

Epoch 4/5
1181/1181 val_acc: 0.875 - val_loss: 0.44243270807327234

Epoch 5/5
1181/1181 val_acc: 0.878095238095238 - val_loss: 0.41822121867097467
